In [ ]:
import torch
print(torch.cuda.is_available(), torch.cuda.device_count())
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import glob
import matplotlib.pyplot as plt
import shutil
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda")
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from IPython.core.debugger import set_trace

In [ ]:
import numpy as np
import os
import skimage.io
import matplotlib.pyplot as plt
from cellpose import models, core
import pandas as pd
from scipy.stats import skew
import glob
from PIL import Image, ImageDraw,ImageFont
from skimage import img_as_ubyte
from skimage.draw import disk
from skimage import measure


In [ ]:
import Granularity_cellprofiler as GP
import glob

In [ ]:
data_dir = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/class1/'
files = glob.glob(data_dir + '*tif')
GP.ImageExploration(image_list = files[:3],diameter_cp = 100).getImageDetails()

In [ ]:
GP.ImageExploration(image_list = files[:3],diameter_cp = 100).getMeanStd()

In [ ]:
GP.ImageExploration(image_list = files[:3],diameter_cp = 100).getCellPerImage()

In [ ]:
import numpy as np
np.linspace(2, 60, 10, endpoint=True, dtype=int)

In [1]:
# create a kernel lost video
import Granularity_cellprofiler as GP
import glob
data_dir = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/class1/'
files = glob.glob(data_dir + '*tif')
outFolder = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/videos'
GP.ImageExploration(image_list = files[:3],diameter_cp = 100).kernelLossVideo(pathout = outFolder,start_kernel = 2 , end_karnel = 20, kernel_size=10,ci = 1)

In [ ]:
# Load the kernel video
from IPython.display import HTML
from base64 import b64encode

# Load video
video_path = os.path.join(outFolder,'kernelLoss.avi')
video_file = open(video_path, "rb").read() 

# Convert video to base64
video_encoded = b64encode(video_file).decode('ascii')

# Create HTML string and display video
HTML(data=f'''
<video alt="test" controls>
<source src="data:video/avi;base64,{video_encoded}" type="video/avi">
</video>''')

In [ ]:
import Granularity_cellprofiler as GP
import glob
data_dir = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/class1/'
files = glob.glob(data_dir + '*tif')
path = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/zoomInImage'
GP.Granularity(image_list = files, pathOrig = path, meanInten = 960,stdInten = 225, kernel =20, windowSize = 3, rnadomWindows = 7).granCalc()

In [ ]:
import Granularity_cellprofiler as GP
import glob
import os

In [ ]:
data_dir = glob.glob('/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/zoomInImage/*')
PathimageName = list(set(data_dir))
imageName = [os.path.basename(path) for path in PathimageName]
origFolder = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/zoomInImage/'
outFolder = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/videos'
GP.ImageToVideo(origFolder = origFolder,imageNames = imageName ,outFolder = outFolder)
#for name in imageName.keys()
#glob.glob(data_dir + 'ZoomInImages/*/ImageSplit/*tif')

In [ ]:
from tifffile import imread
import cv2
import os
import numpy as np
from PIL import Image
import glob

data_dir = glob.glob('/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/ZoomInImages/*')
PathimageName = list(set(data_dir))
imageName = [os.path.basename(path) for path in PathimageName]
origFolder = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/ZoomInImages/'
outFolder = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/videos'

In [ ]:
files = glob.glob(origFolder + imageName[0] + '/ImageSplit/*tif')
frame = Image.open(files[0])
frame = np.array(frame)

In [ ]:
frame.shape

In [ ]:
def images_to_video(image_folder, video_name,outFolder, frame_rate):
    images = [img for img in os.listdir(image_folder) if img.endswith(".tif")]
    # Sort the images by name
    images.sort()

    # Load the first image and get its dimensions
    frame = Image.open(os.path.join(image_folder, images[0]))
    frame = np.array(frame)
    
    height, width = frame.shape

    # Initialize the video writer
    video = cv2.VideoWriter(os.path.join(outFolder,video_name), cv2.VideoWriter_fourcc(*'DIVX'), frame_rate, (width,height), isColor=False)

    # Iterate over all images
    for image in images:
        # Open the image and append it to the video
        frame = Image.open(os.path.join(image_folder, image))
        frame = np.array(frame)
        
        # Since the color depth of the image and the video may differ, we normalize the images to 8 bit before adding them to the video
        frame = ((frame - frame.min()) / (frame.ptp()) * 255).astype(np.uint8)
        
        # Write the frame to the video
        video.write(frame)

    # Release the video file
    video.release()

In [ ]:
image_folder = origFolder + imageName[0] + '/ImageSplit/'
images = [img for img in os.listdir(image_folder) if img.endswith(".tif")]
frame = Image.open(os.path.join(image_folder, images[0]))
frame = np.array(frame)
height, width = frame.shape
video = cv2.VideoWriter(os.path.join(outFolder,'output_video1.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 30, (width,height), isColor=False)
video.write(frame)
for image in images:
    frame = Image.open(os.path.join(image_folder, image))
    frame = np.array(frame)
    frame = ((frame - frame.min()) / (frame.ptp()) * 255).astype(np.uint8)
    video.write(frame)
video.release()

In [ ]:
def rgbImge(image_input):
    input_gs_image = (image_input / image_input.max()) * 255
    ch2_u8 = np.uint8(input_gs_image)
    rgb_input_img = np.zeros((np.shape(ch2_u8)[0], np.shape(ch2_u8)[1], 3), dtype=np.uint8)
    rgb_input_img[:, :, 0] = ch2_u8
    rgb_input_img[:, :, 1] = ch2_u8
    rgb_input_img[:, :, 2] = ch2_u8
    pilimage = Image.fromarray(rgb_input_img)
    return rgb_input_img

In [ ]:
from skimage.io import imread
from skimage import io
from skimage.util import img_as_ubyte

image_folder = origFolder + imageName[0] + '/ImageSplit/'
images = [img for img in os.listdir(image_folder) if img.endswith(".tif")]
frame = io.imread(os.path.join(image_folder, images[0]))
# frame = rgbImge(frame)
# width,height,layers = frame.shape
# video = cv2.VideoWriter(os.path.join(outFolder,'output_video3.avi'), cv2.VideoWriter_fourcc(*'MJPG'),10.0,(width,height),  isColor = True)
# for image in images:
#     frame = io.imread(os.path.join(image_folder, image))
#     frame = rgbImge(frame)
#     #frame = ((frame - frame.min()) / (frame.ptp()) * 255).astype(np.uint8)
#     video.write(frame)
# video.release()

In [ ]:
len(images)

In [ ]:
plt.imshow(frame*10)

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Load video
video_path = os.path.join(outFolder,'output_video3.avi')
video_file = open(video_path, "rb").read() 

# Convert video to base64
video_encoded = b64encode(video_file).decode('ascii')

# Create HTML string and display video
HTML(data=f'''
<video alt="test" controls>
<source src="data:video/avi;base64,{video_encoded}" type="video/avi">
</video>''')

In [ ]:
images = [img for img in os.listdir(image_folder) if img.endswith(".tif")]
frames = []
for image in images:
    frame_temp = io.imread(os.path.join(image_folder, image))
    frame_temp = rgbImge(frame_temp)
    frames.append(frame_temp)

In [ ]:
from io import BytesIO
byteframes = []
for img in frames:
    byte = BytesIO()
    byteframes.append(byte)
    img.save(byte, format="GIF")
gifs = [Image.open(byteframe) for byteframe in byteframes]
gifs[0].save("good.gif", append_images=gifs[1:])

In [ ]:
frame = imread(os.path.join(image_folder, images[0]))
frame = img_as_ubyte(frame)

In [ ]:
origFolder + imageName[0] + '/ImageSplit/'

In [ ]:
images_to_video(origFolder + imageName[0] + '/ImageSplit/',outFolder, 'output_video.avi', 30)

In [ ]:
!pwd

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4_file = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/videos/UHqjfLR.mp4'

# load video
video = open(mp4_file, "rb").read()

# encode to base64
video_encoded = b64encode(video).decode("ascii")

# create HTML string
video_tag = '<video controls alt="test" src="data:video/mp4;base64,{0}">'.format(video_encoded)

# display video
HTML(data=video_tag)

In [ ]:
import os

In [ ]:
os.path.join

In [ ]:
image = skimage.io.imread(os.path.join(data_dir,files[2]))
image.shape

In [ ]:
minAxsis = min(image.shape[0],image.shape[1])
winSize = 6
while minAxsis % winSize != 0:
    minAxsis -= 1
stepSize =  int(minAxsis/winSize)
print(f'max image size: {minAxsis}, step size: {stepSize}')

In [ ]:
for i in range(0,minAxsis,stepSize):
    print(f'{i}')

In [ ]:
imageSlice = []
for i in range(0,minAxsis,stepSize):
    for j in range(0,minAxsis,stepSize):
        imageSlice.append(image[i:i+stepSize,j:j+stepSize])
        

In [ ]:
range_xy = 0+stepSize,minAxsis-250
np.random.randint(low = range_xy[0],high = range_xy[1],size =1).tolist()[0]

In [ ]:
rnadomWindows = [1]*3
rnadomWindows.pop()

In [ ]:
rnadomWindows

In [ ]:
y = minAxsis
x = 0
imageSlice = []
for i in range(0,minAxsis,stepSize):
    for j in range(stepSize):
        imageSlice = image[x[i]

In [ ]:
for i in range(0,minAxsis,stepSize):
    print(f'{i}')

In [ ]:
np.std(image)

In [ ]:
for i,im in enumerate(imageSlice):
    i

In [ ]:
import cv2
import glob
import numpy as np
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  

In [ ]:
data_dir = '/data/kanferg/Images/022419_catGFP'
files = glob.glob("/data/kanferg/Images/022419_catGFP/" + "*tif")
image = skimage.io.imread(os.path.join(data_dir,files[2]))
channel_1 = image[0,:,:]

In [ ]:
from skimage import exposure

p2, p98 = np.percentile(channel_1, (40, 98))
channel_1_rescaled = exposure.rescale_intensity(channel_1, in_range=(p2, p98))

plt.imshow(channel_1_rescaled, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
open_vec = np.linspace(2, 20, 10, endpoint=True, dtype=int)
open_vec

In [ ]:
for opning in open_vec:
    openImage = self.openingOperation(kernel = opning)
    tableGran_temp = self.measure_properties(input_image =openImage,input_mask = self.mask ,prop_names = fetureLabel)
    tableColumn[opning] = tableGran_temp['mean_int'].tolist()

In [ ]:
from skimage import measure, morphology
# openingOperation:
selem = morphology.disk(kernel, dtype=bool)
eros_pix = morphology.erosion(image, footprint=selem)
imageOpen = morphology.dilation(eros_pix, footprint=selem)

In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure, morphology

In [ ]:
def openingOperation(image, kernel):
    selem = morphology.disk(kernel, dtype=bool)
    eros_pix = morphology.erosion(image, footprint=selem)
    imageOpen = morphology.dilation(eros_pix, footprint=selem)
    return imageOpen
kernels = np.linspace(2, 20, 10, endpoint=True, dtype=int)
images = [openingOperation(image[0,:,:], kernel) for kernel in kernels]

In [ ]:
# from skimage import exposure
# def normalize_and_enhance(image):
#     image = (image - np.min(image)) / (np.max(image) - np.min(image))
#     image = exposure.equalize_adapthist(image, clip_limit=0.3)
#     return image
# enhanced_images = [normalize_and_enhance(image) for image in images]

In [ ]:
# def normalize_16bit(image):
#     return (image - np.min(image)) / (np.max(image) - np.min(image))
# normalized_images = [normalize_16bit(image) for image in images]

In [ ]:
# Remove axes for clarity
fig, ax = plt.subplots()

ax.axis('off')

# Display the first frame
im = ax.imshow(images[0]*10, cmap='gray')

# Function to update figure
def update(i):
    # Update the image data
    im.set_array(images[i]*10)

# Create animation
ani = FuncAnimation(fig, update, frames=len(images), interval=500) # 500ms between frames
HTML(ani.to_jshtml())

In [ ]:
# save images of Cat-GFP has single

In [ ]:
# from IPython.display import clear_output

# clear_output(wait=True) # remove all display 
# data_dir = '/data/kanferg/Images/022419_catGFP'
# #files = glob.glob("/data/kanferg/Images/022419_catGFP/" + "*tif")
# files = os.listdir(data_dir)
# files = [f for f in files if f.endswith('.tif')]
# target_dir = "022419_catGFP"
# for i in range(len(files)):
#     image = skimage.io.imread(os.path.join(data_dir,files[i]))
#     channel_1 = image[0,:,:]
#     try:
#         skimage.io.imsave(os.path.join(target_dir,str(i)+'.tif'), channel_1)
#     except:
#         continue
#     print('-' * i, end = '\r')    

In [ ]:
data_dir = "022419_catGFP/class1"

In [ ]:
# dataset = ImageFolder(target_dir,transform = transforms.Compose([
#     transforms.ToTensor()
# ]))
# img, label = dataset[0]

# print(img.shape,label) 
# print("Follwing classes are there : \n",dataset.classes)

In [ ]:
# def display_img(img,label):
#     print(f"Label : {dataset.classes[label]}")
#     plt.imshow(img)
# '''
# In the function `img.permute(1,2,0)`, `permute(1,2,0)` is used to change the dimensions of the `img` object.

# The `img` object has dimensions `torch.Size([3, 150, 150])`, which means it has 3 channels (RGB) and an image size of 150x150 pixels.
# '''
# #display the first image in the dataset
# display_img(*dataset[1])

In [ ]:
 # remove all display 
data_dir = '/data/kanferg/Images/022419_catGFP/class1'
files = os.listdir(data_dir)
files = [f for f in files if f.endswith('.tif')]
target_dir = "022419_catGFP"
for i in range(len(files)):
    image = skimage.io.imread(os.path.join(data_dir,files[i]))
    channel_1 = image[0,:,:]
    try:
        skimage.io.imsave(os.path.join(target_dir,str(i)+'.tif'), channel_1)
    except:
        continue
    print('-' * i, end = '\r')    

In [ ]:
data_dir = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/class1/'
files = glob.glob(data_dir + '*tif')
image = skimage.io.imread(os.path.join(data_dir,files[2]))
image.shape, image.dtype

In [ ]:
import random


In [ ]:
data_dir = '/data/kanferg/Images/Pex_project/Subrah_2024/022419_catGFP/class1/'
files = glob.glob(data_dir + '*tif')
files[:10]

In [ ]:
x = [skimage.io.imread(image) for image in files[:10]]
len(x)
np.choose(5,np.arange(len(x)))

In [ ]:
from cellpose import models, core
from AIPyS import AIPS_file_display as afd

In [ ]:
use_GPU = core.use_gpu()
model = models.Cellpose(gpu=use_GPU, model_type="cyto")

In [ ]:
30/110

In [ ]:
mask, flows, styles, diams = model.eval(x[0]*50, diameter=100, flow_threshold=None, channels=[0,0])

In [ ]:
plt.imshow( mask, cmap=plt.cm.gray) 

In [ ]:
zz = [1,32,45]
np.mean(zz)

In [ ]:
maskContour = afd.Compsite_display(input_image = x[0], mask_roi = mask, channel= 0).draw_ROI_contour()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 8)) 
ax[0].imshow( x[0], cmap=plt.cm.gray) 
ax[0].title.set_text('GFP Catalase') 
ax[1].imshow(maskContour, cmap=plt.cm.rainbow) 
ax[1].title.set_text('Mask - Target') 

In [ ]:
plt.imshow(x[0][600:1200,350:850]*5, cmap=plt.cm.gray) 

In [ ]:
styles

In [ ]:
data_dir = '/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117'
files = glob.glob("/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117/" + "*tiff")
image = skimage.io.imread(os.path.join(data_dir,files[0]))
image.shape

In [ ]:
def draw_ROI_contour(input_image):
    input_gs_image = input_image
    input_gs_image = (input_gs_image / input_gs_image.max()) * 255
    ch2_u8 = np.uint8(input_gs_image)
    rgb_input_img = np.zeros((np.shape(ch2_u8)[0], np.shape(ch2_u8)[1], 3), dtype=np.uint8)
    rgb_input_img[:, :, 0] = ch2_u8
    rgb_input_img[:, :, 1] = ch2_u8
    rgb_input_img[:, :, 2] = ch2_u8
    return rgb_input_img

In [ ]:
draw_ROI_contour(image[:,:,0)

In [ ]:
image = np.transpose(image, (2, 0, 1))
image.dtype

In [ ]:
PIL_image = Image.fromarray(np.uint8(image[:,:,1))
plt.imshow(PIL_image,cmap='binary')

In [ ]:
data_dir = '/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117'
dataset = ImageFolder(data_dir,transform = transforms.Compose([transforms.ToTensor()]))
def display_img(img,label):
    print(f"Label : {dataset.classes[label]}")
    plt.imshow(img.permute(1,2,0))
'''
In the function `img.permute(1,2,0)`, `permute(1,2,0)` is used to change the dimensions of the `img` object.

The `img` object has dimensions `torch.Size([3, 150, 150])`, which means it has 3 channels (RGB) and an image size of 150x150 pixels.
'''
#display the first image in the dataset
display_img(*dataset[1])

In [ ]:
os.getcwd()

In [ ]:
os.chdir("/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117")

In [ ]:
os.chdir("/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117")


In [ ]:
os.mkdir("class_fake_val")

In [ ]:
files = glob.glob("/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117/" + "*tiff")

In [ ]:
target_dir = "/data/kanferg/Images/96_well_40xW_BAf_CQ_VPS34i_073021/20210730_113819_117/" + "seg_train/PEX"
for i in range(0,100):
    shutil.copy(files[i], target_dir)

In [ ]:
target_dir